In [ ]:
pip install langchain faiss-cpu beautifulsoup4 requests langchain-upstage

In [111]:
import os
import requests
from bs4 import BeautifulSoup
from langchain_upstage import UpstageEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import CharacterTextSplitter
from langchain_upstage import ChatUpstage
from langchain.chains import RetrievalQA
from langchain.chains.question_answering import load_qa_chain

os.environ["UPSTAGE_API_KEY"] = "up_coecXafSJVG1v17EEZ3lxjFbZ8xcD"

def extract_text_from_url(urls):
    all_texts = []

    for url in urls:
        try:
            response = requests.get(url)
            soup = BeautifulSoup(response.text, 'html.parser')

            paragraphs = soup.find_all('p')
            text = "\n".join([para.get_text() for para in paragraphs])

            all_texts.append(text)
        except Exception as e:
            print(f"Error processing {url}: {e}")

    return all_texts

now_number = 28225
urls = []

for number in range(now_number-30, now_number+1):
  urls.append("https://cse.knu.ac.kr/bbs/board.php?bo_table=sub5_1&wr_id=" + str(number))

document_text = extract_text_from_url(urls)

if isinstance(document_text, list):
    texts = []
    for doc in document_text:
        if isinstance(doc, str):
            texts.extend(text_splitter.split_text(doc))
        else:
            raise TypeError("Each document in the list must be a string")
else:
    raise TypeError("document_text must be a list of strings")

embedding_model = UpstageEmbeddings(
    model="solar-embedding-1-large"
)

vectorstore = FAISS.from_texts(texts, embedding_model)

upstage_llm = ChatUpstage(api_key=os.getenv("UPSTAGE_API_KEY"))

qa_chain = load_qa_chain(llm=upstage_llm, chain_type="stuff")

qa = RetrievalQA(combine_documents_chain=qa_chain, retriever=vectorstore.as_retriever())

In [112]:
query = "TOPCIT 등록기간이 언제인가요?"
print(qa.run(query))

TOPCIT 등록기간은 2024년 9월 9일(월)부터 9월 12일(목)까지입니다.


In [113]:
query = "튜터 모집 기간은 어떻게 되나요?"
print(qa.run(query))

튜터 모집 기간은 다음과 같습니다.

1차~5차 신청: 2024. 9. 19.(목) 오전까지
2차 모집: 2024. 9. 23.(월) 10시까지

해당 기간 내에 구글 폼을 통해 신청하실 수 있습니다.
